# 0) Warmup

In [5]:
!ls groupby_and_aggregation

Groupby.ipynb            large_countries_2015.csv split_apply_combine.png


In [44]:
import pandas as pd
import numpy as np

df = pd.read_csv('./large_countries_2015.csv', index_col=0)

df['population'] = df['population'] / 1000000
df['population'] = round(df['population'], 1)

In [7]:
df.head()

,population,fertility,continent
Bangladesh,161.0,2.12,Asia
Brazil,207.8,1.78,South America
China,1376.0,1.57,Asia
India,1311.1,2.43,Asia
Indonesia,257.6,2.28,Asia


## 0.1) Calculate the average population size of countries in the dataset

In [10]:
print(f'The average population size of large countries in 2015 was {round(df.population.mean(), 2)} million')

The average population size of large countries in 2015 was 375.35 million


## 0.2) Calculate the average fertility rate in the dataset

In [19]:
# If no person dies, 
# what would be the number of people living in large countries in the next generation?
df['pop_next_gen'] = df['population'] + df['population'] * 0.5 * df['fertility']
df.head()

,population,fertility,continent,pop_next_gen
Bangladesh,161.0,2.12,Asia,331.6600
Brazil,207.8,1.78,South America,392.7420
China,1376.0,1.57,Asia,2456.1600
India,1311.1,2.43,Asia,2904.0865
Indonesia,257.6,2.28,Asia,551.2640


In [21]:
population = round(df['pop_next_gen'].sum(), 2)
print(f'The sum of people living in large countries in the next generation is {population}')

The sum of people living in large countries in the next generation is 9459.89


In [22]:
# The average fertility rate would be the fertility rate that would lead to exactly this number
# if we applied it to every country
df['pop_next_arithmetic_mean'] = df['population'] + df['population'] * 0.5 * df['fertility'].mean()
df.head()

,population,fertility,continent,pop_next_gen,pop_next_arithmetic_mean
Bangladesh,161.0,2.12,Asia,331.6600,357.218750
Brazil,207.8,1.78,South America,392.7420,461.056250
China,1376.0,1.57,Asia,2456.1600,3053.000000
India,1311.1,2.43,Asia,2904.0865,2909.003125
Indonesia,257.6,2.28,Asia,551.2640,571.550000


In [23]:
population_arithmetic_mean = round(df['pop_next_arithmetic_mean'].sum(), 2)
print(f'The sum of people living in large countries in the next generation is {population_arithmetic_mean}')

The sum of people living in large countries in the next generation is 9993.69


In [24]:
# Taking the arithmetic mean does not provide us with the right number of people in the next 
# generation
# The reason is that the influence of countries du to differences in population size are not 
# taken into account by the arithmetic mean

In [27]:
# Weighted arithmetic mean
# Weights will be the population size
weighted_average_fertility = np.average(df.fertility, weights=df.population)

In [30]:
df['pop_next_weighted_arithmetic_mean'] = df['population'] + df['population'] * 0.5 * weighted_average_fertility
df.head()

,population,fertility,continent,pop_next_gen,pop_next_arithmetic_mean,pop_next_weighted_arithmetic_mean
Bangladesh,161.0,2.12,Asia,331.6600,357.218750,338.138354
Brazil,207.8,1.78,South America,392.7420,461.056250,436.429503
China,1376.0,1.57,Asia,2456.1600,3053.000000,2889.927794
India,1311.1,2.43,Asia,2904.0865,2909.003125,2753.622333
Indonesia,257.6,2.28,Asia,551.2640,571.550000,541.021366


In [31]:
population_weighted_arithmetic_mean = round(df['pop_next_weighted_arithmetic_mean'].sum(), 2)
print(f'The sum of people living in large countries in the next generation is {population_weighted_arithmetic_mean}')

The sum of people living in large countries in the next generation is 9459.89


Now imagine that we are interested in the averages per continent. How can we do that?

# 1) Groupby and aggregation

Today we are going to talk about 3 different but related topics.

1. Aggregate functions in pandas
2. The groupby method of a pandas.DataFrame
3. Applying functions or transformations to pandas.Series

## 1.1) Aggregate functions in pandas

**Aggregate function**: takes multiple rows as input and returns a single value

In [ ]:
df.mean()

In [ ]:
df[['population', 'fertility']].sum()

In [ ]:
df.count()

In [ ]:
df.max()

In [ ]:
df.min()

In [ ]:
df.median()

In [ ]:
df.quantile(.1)

In [ ]:
df.prod()

In [ ]:
df.std()

In [ ]:
df.var()

In [33]:
df.describe()

,population,fertility,pop_next_gen,pop_next_arithmetic_mean,pop_next_weighted_arithmetic_mean
count,12.000000,12.000000,12.000000,12.000000,12.000000
mean,375.350000,2.437500,788.324417,832.807813,788.324417
std,456.517642,1.200781,903.267856,1012.898518,958.795800
min,100.700000,1.450000,218.385000,223.428125,211.493989
25%,139.375000,1.737500,261.445625,309.238281,292.720702
50%,185.550000,2.125000,434.385000,411.689062,389.699202
75%,273.650000,2.567500,658.774500,607.160937,574.730189
max,1376.000000,5.890000,2904.086500,3053.000000,2889.927794


In [32]:
# apply a customized set of aggregate functions
df.agg(['mean', 'var'])

,population,fertility,pop_next_gen,pop_next_arithmetic_mean,pop_next_weighted_arithmetic_mean
mean,375.350000,2.437500,788.324417,8.328078e+02,788.324417
var,208408.357273,1.441875,815892.819068,1.025963e+06,919289.386805


In [34]:
# Aggregation with different functions for different columns
df.agg({'population': 'mean', 'fertility': 'var'})

population    375.350000
fertility       1.441875
dtype: float64

## 1.2) Group By in pandas

Data Aggregation in Python is very closely linked to the ``DataFrame.groupby()`` statement.

- Splitting the data into groups based on some criteria.
- Applying a(n aggregate) function to each group independently.
- Combining the results into a data structure.

![Split-Apply-Combine](split_apply_combine.png)

## 1.2.1) Splitting

In [46]:
df = df[['population', 'fertility', 'continent']]

In [47]:
# Inspect the type of groupby
type(df.groupby('continent'))

pandas.core.groupby.generic.DataFrameGroupBy

In [48]:
# Assign the groupby object to a variable name
g = df.groupby('continent')

In [49]:
# Inspect the attributes and methods of the object
g.groups

{'Africa': ['Nigeria'], 'Asia': ['Bangladesh', 'China', 'India', 'Indonesia', 'Japan', 'Pakistan', 'Philippines'], 'Europe': ['Russia'], 'North America': ['Mexico', 'United States'], 'South America': ['Brazil']}

In [ ]:
[('Africa', df_africa),
 ('Asia', df_asia),
 (...),
 ...]

In [50]:
# iterrate over the DataFrames
for group, df_group in g:
    print(group)
    print(df_group)
    print('\n')

Africa
         population  fertility continent
Nigeria       182.2       5.89    Africa


Asia
             population  fertility continent
Bangladesh        161.0       2.12      Asia
China            1376.0       1.57      Asia
India            1311.1       2.43      Asia
Indonesia         257.6       2.28      Asia
Japan             126.6       1.45      Asia
Pakistan          188.9       3.04      Asia
Philippines       100.7       2.98      Asia


Europe
        population  fertility continent
Russia       143.5       1.61    Europe


North America
               population  fertility      continent
Mexico              127.0       2.13  North America
United States       321.8       1.97  North America


South America
        population  fertility      continent
Brazil       207.8       1.78  South America




In [52]:
# get the DataFrame of a specific group
g.get_group('Asia')

,population,fertility,continent
Bangladesh,161.0,2.12,Asia
China,1376.0,1.57,Asia
India,1311.1,2.43,Asia
Indonesia,257.6,2.28,Asia
Japan,126.6,1.45,Asia
Pakistan,188.9,3.04,Asia
Philippines,100.7,2.98,Asia


There are different ways of splitting a ``pandas.DataFrame``

In [ ]:
# 1. By one column
df.groupby('continent')

In [54]:
# 2. By two (or more) columns
df.groupby(['continent', 'population'])

In [58]:
# Apply a aggregate function to it
df.groupby('continent')['population'].mean()

continent
Africa           182.200000
Asia             503.128571
Europe           143.500000
North America    224.400000
South America    207.800000
Name: population, dtype: float64

### More sophisticated applications

``.apply()`` and ``.transform()``

``.transform()`` takes a ``pd.DataFrame`` or a ``pd.Series`` as input, applies a specified function to each ``pd.Series`` it receives and returns a ``pd.DataFrame`` or ``pd.Series`` of equal shape.
> **input:** ``pd.DataFrame`` or ``pd.Series``<br>
> **operates on:** each ``pd.Series``separately<br>
> **output:** ``pd.DataFrame`` or ``pd.Series`` of the same shape

In [64]:
g = df.groupby('continent')

In [66]:
g['population'].mean()

continent
Africa           182.200000
Asia             503.128571
Europe           143.500000
North America    224.400000
South America    207.800000
Name: population, dtype: float64

In [67]:
g['population'].transform('mean')

Bangladesh       503.128571
Brazil           207.800000
China            503.128571
India            503.128571
Indonesia        503.128571
Japan            503.128571
Mexico           224.400000
Nigeria          182.200000
Pakistan         503.128571
Philippines      503.128571
Russia           143.500000
United States    224.400000
Name: population, dtype: float64

In [68]:
# Transform can be incredibly powerful to create new columns - or in order to fill in missing values with .fillna()
df['continent_population'] = g['population'].transform('sum')
df.head()

,population,fertility,continent,continent_population
Bangladesh,161.0,2.12,Asia,3521.9
Brazil,207.8,1.78,South America,207.8
China,1376.0,1.57,Asia,3521.9
India,1311.1,2.43,Asia,3521.9
Indonesia,257.6,2.28,Asia,3521.9


In [69]:
df['weighted_fertility'] = df['fertility'] * df['population'] / df['continent_population']

In [70]:
df.head()

,population,fertility,continent,continent_population,weighted_fertility
Bangladesh,161.0,2.12,Asia,3521.9,0.096914
Brazil,207.8,1.78,South America,207.8,1.780000
China,1376.0,1.57,Asia,3521.9,0.613396
India,1311.1,2.43,Asia,3521.9,0.904618
Indonesia,257.6,2.28,Asia,3521.9,0.166765


In [71]:
df.groupby('continent')['weighted_fertility'].sum()

continent
Africa           5.890000
Asia             2.082073
Europe           1.610000
North America    2.015276
South America    1.780000
Name: weighted_fertility, dtype: float64

``.apply()`` takes a ``pd.DataFrame`` or a ``pd.Series`` as input, applies a specified function, either to the ``pd.DataFrame`` or to each ``pd.Series``. It returns, depending on the specified funtion, a ``pd.DataFrame`` of flexible size, a ``pd.Series`` or a scalar.
> **input:** ``pd.DataFrame``<br>
> **operates on:** the whole ``pd.DataFrame`` or each ``pd.Series``, depending on the function<br>
> **output:** ``pd.DataFrame``, ``pd.Series`` or scalar

In [73]:
def weighted_average(df):
    '''Calculates the weighted average fertility'''
    weights = df['population']/df['population'].sum()
    weighted_fertility = df['fertility'] * weights
    weighted_average_fertility = weighted_fertility.sum()
    return weighted_average_fertility

In [74]:
weighted_average(df)

2.2004764442076286

In [75]:
df.groupby('continent').apply(weighted_average) # .groupby().apply(name_of_function)

continent
Africa           5.890000
Asia             2.082073
Europe           1.610000
North America    2.015276
South America    1.780000
dtype: float64

### Combine

Nothing to do here. Pandas is doing that for us.